# Ejemplos

Un conjunto de $ n=50 $ observaciones y $ 25 $ parámetros. Se agrupan por tipo de error introducido.

In [127]:
import numpy as np;
import matplotlib.pyplot as plt;
from scipy.stats import beta;
from scipy.stats import uniform;
from scipy.stats import chi;
from scipy.stats import gamma;
from scipy.stats import lognorm;


In [128]:
numSamples    = 50;
numParameters = 25;
numGroups     = 5;
lowBound      = -5;
highBound     = 5;
onesVector    = np.ones(numSamples);
lowBeta       = -15;
highBeta      = 25;
betaV         = np.random.randint(lowBeta, highBeta, numParameters+1)

X = np.random.uniform(low=lowBound, high=highBound, size=(numSamples, numParameters))
X = np.vstack((onesVector, X.T)).T

dictResponseY = {}
realY = X @ betaV
#print("real ", realY)
minY = np.min(realY)
maxY = np.max(realY)
#print(minY)
#print(maxY)

modelUniform = uniform(minY*0.0001, maxY*0.005);
modelBeta    = beta(np.absolute(minY*0.8), np.absolute(maxY*0.8));
modelChi     = chi(np.absolute((minY + maxY)*0.45));
modelGamma   = gamma(np.absolute((minY + maxY)*0.05));
modelLogNorm = lognorm(0.6);

dictResponseY["uniforme"]  = modelUniform.rvs(size=numSamples);
dictResponseY["beta"]      = modelBeta.rvs(size=numSamples);
dictResponseY["chi"]       = modelChi.rvs(size=numSamples);
dictResponseY["gamma"]     = modelGamma.rvs(size=numSamples);
dictResponseY["logNormal"] = modelLogNorm.rvs(size=numSamples);

#print("dictResponseY ", dictResponseY)

dictResponseY["uniforme"]  += realY;
dictResponseY["beta"]      += realY;
dictResponseY["chi"]       += realY;
dictResponseY["gamma"]     += realY;
dictResponseY["logNormal"] += realY;

#print("dictResponseY sum ", dictResponseY)


# Regresión lineal múltiple

$$ y_i = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_p x_p + \epsilon $$

##### El modelo

$$ \bf{\hat{Y}} = \begin{bmatrix} 1 & x_{1,1} & \cdots & x_{1,p} \\ 1 & x_{2,1} & \cdots & x_{2,p}  \\ \vdots & \vdots& \ddots &\vdots \\ 1 & x_{n,1} & \cdots & x_{n,p}   \end{bmatrix} \begin{bmatrix} \beta_0 \\ \beta_1 \\ \vdots \\ \beta_n \end{bmatrix} = \begin{bmatrix} \hat{y_1} \\ \hat{y_2} \\ \vdots \\ \hat{y_n} \end{bmatrix} = \bf{X} \bf{\beta}$$

$$ \bf{\hat{\beta}} = (\bf{X}^T \bf{X})^{-1}\bf{X}^T\bf{y}$$

### Criterio de mínimos cuadrados

Forma de medir la cercanía de la la línea con el conjunto de datos.

#### El residual

$$ e_i = y_i - \hat{y_i} $$

- $ y_i $ valor de $ i $-ésima respuesta observada $ \hat{y}_i $ es el valor de la respuesta $ i $-ésima predicha por el modelo.

#### Suma residual de cuadrados (RSS)

$$ RSS = e_1^2 + e_2^2 + \cdots + e_n^2 $$


### Problema de optimización

**El obejtivo es estimar los parámetros $ \hat{\beta_0} $ y $ \hat{\beta_1} $ para minimizar el RSS**

$$ \bf{e(\bf{\beta})} = \frac{1}{2}(\bf{y} - \bf{X} \bf{\beta})^T (\bf{y} - \bf{X} \bf{\beta}) $$

$$ \bf{\hat{\beta}} = \bf{\beta^*} = arg\ \underset{\bf{\beta}}{min} \ \bf{e(\beta)} = arg\ \underset{\bf{\beta}}{min} \frac{1}{2}(\bf{y} - \bf{X} \bf{\beta})^T (\bf{y} - \bf{X} \bf{\beta}) $$

Se encuentra el mínimo a través de la derivada.

$$ \frac{d \bf{e(\bf{\beta})}}{d\bf{\beta}} = 0 $$

$$ \frac{d \bf{e(\bf{\beta})}}{d\bf{\beta}} = \frac{d \bf{\frac{1}{2} (\bf{y}^T\bf{y} - \bf{y}^T\bf{X}\bf{\beta} - (\bf{X}\bf{\beta})^T\bf{y} + (\bf{X}\bf{\beta})^T \bf{X}\bf{\beta} )}}{d\bf{\beta}} $$

$$ \frac{d \bf{e(\bf{\beta})}}{d\bf{\beta}} = \frac{1}{2} (-2 \bf{y}^T\bf{X} + 2\bf{X}^T \bf{X}\bf{\beta} )= \bf{0}$$

$$ \bf{\hat{\beta}} = (\bf{X}^T \bf{X})^{-1}\bf{X}^T\bf{y}$$

In [129]:
import pandas as pd;

nombre_resul =  'Resultados_data.xlsx';

writer = pd.ExcelWriter(nombre_resul, engine='xlsxwriter')

tabla_uniforme = pd.DataFrame(data=np.c_[dictResponseY["uniforme"],
                                         dictResponseY["beta"],
                                         dictResponseY["chi"],
                                         dictResponseY["gamma"],
                                         dictResponseY["logNormal"]],
                                         columns=["Y uniforme", "Y beta",
                                                  "Y chi", "Y gamma", "Y logNormal"],
                              index=np.arange(len(dictResponseY["uniforme"]))+1)

tabla_X     = pd.DataFrame(X, columns=list(range(len(X[0, :]))))
tabla_beta  = pd.DataFrame(betaV, columns=["beta"], index=np.arange(len(betaV))+1)
tabla_realY = pd.DataFrame(realY, columns=["Yreal"], index=np.arange(len(realY))+1)

tabla_uniforme.to_excel(writer, sheet_name='Respuestas')
tabla_X.to_excel(writer, sheet_name='X')
tabla_beta.to_excel(writer, sheet_name='beta')
tabla_realY.to_excel(writer, sheet_name='Y real')

writer.save()
print(f'\n\nEn el archivo "{nombre_resul}" se guardaron los resultados.')




En el archivo "Resultados_data.xlsx" se guardaron los resultados.


In [130]:
df         = pd.read_excel(nombre_resul, sheet_name=None)
Ydata      = df["Respuestas"].to_numpy()[:, 1:]
Xdata      = df["X"].to_numpy()[:, 1:]
Yreal      = df["Y real"].to_numpy()[:, 1:]
betaVector = df["beta"].to_numpy()[:, 1:]
#print(betaVector)
#print(np.shape(betaVector))

In [131]:
listBetaRespuesta = [];

rowsY, columY = np.shape(Ydata)

for Y_i in range(columY):
    beta = np.linalg.inv(Xdata.T @ Xdata) @ Xdata.T @ Ydata[:, Y_i]
    listBetaRespuesta.append(beta)
    #print("beta ", beta)

nombre_resul =  'Resultados_beta.xlsx';

writer = pd.ExcelWriter(nombre_resul, engine='xlsxwriter')

tabla_beta = pd.DataFrame(data=np.c_[listBetaRespuesta[0],
                                     listBetaRespuesta[1],
                                     listBetaRespuesta[2],
                                     listBetaRespuesta[3],
                                     listBetaRespuesta[4]],
                                     columns=["beta uniforme", "beta beta",
                                                  "beta chi", "beta gamma", "beta logNormal"],
                              index=np.arange(len(listBetaRespuesta[0]))+1)

tabla_beta.to_excel(writer, sheet_name='Respuestas betas')

writer.save()
print(f'\n\nEn el archivo "{nombre_resul}" se guardaron los resultados.')





En el archivo "Resultados_beta.xlsx" se guardaron los resultados.
